# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','Speed.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

Plot the definition of before after ...

In [ ]:
def plot_session_def(ax, sessionSlices= tuple()):
    eps=0
    xpos=ax.get_xticks()
    
    for sli in sessionSlices:
        assert isinstance(sli, slice), 'Bad Boy!'        
        stop=sli.stop
        if stop is None:
            stop=-1
        start=sli.start
        if start>=0:
            start+=1
#             stop+=1
        
        ax.axvspan(xmin=start+eps, xmax=stop-eps, zorder=-1, alpha=.8, ec= None,
                  color='xkcd:ivory')
        
    if len(sessionSlices)==3:
        for sli,String in zip(sessionSlices,['Before','Acute','Stable']):
            stop=sli.stop
            if stop is None:
                stop=-1
            start=sli.start
            if start>=0:
                start+=1
            
            mid=(start+stop)/2
            
            ax.text(mid,ax.get_ylim()[1],s=String,
                   fontsize='xx-small', ha='center',va='center')

---

Plot the speed definition graphic

In [ ]:
def plot_speed_def(ax):
    x0=np.arange(0,7,.01)
    y0=10*x0+10

    x1=np.arange(7, 8.16,.01)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)

    y=smooth(y*1.1,sigma=50,mode='constant')

    x40=np.where(np.logical_and(y<=40,x>7))[0][0]
    x60=np.where(np.logical_and(y<=60,x>7))[0][0]
    #plot the traj
    ax.plot(x, y, lw=2, c='navy', )
    #plot the 40 and 60 horizontal lines
    ax.hlines(y=40, xmin=-10, xmax=x[x40], lw=ax.spines['bottom'].get_lw(), ls='--')
    ax.hlines(y=60, xmin=-10, xmax=x[x60],lw=ax.spines['bottom'].get_lw(), ls='--')
    #plot the red piece
    ax.plot([x[x40],x[x60]],[40,60], lw=ax.spines['bottom'].get_lw(), c='r',zorder=4)
    #plot the x-axis at the red piece
    ax.arrow(x[x60],60,1.3,0,lw=ax.spines['bottom'].get_lw(),head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    # plot the curve
    x2=7.5
    y2=y[np.where(x>=x2)[0][0]]
    ax.add_patch(FancyArrowPatch(posA=(x[x60]+.8,62), posB=(x2-.1,y2-2),
                                 arrowstyle='-', connectionstyle='arc3,rad=-.5',color='r',
                                 mutation_scale=15,lw=ax.spines['bottom'].get_lw(),zorder=6))
    #writing 
    ax.text(7.9,40,'speed',rotation=-75, c='r', fontsize='xx-small',ha='left',va='center')
    
    
    ax.set_xticks([0,1,2,3,4,5,6,7,8])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'',20,'',40,'',60,'',80,''])
    ax.spines['bottom'].set_bounds(0,8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.1,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)')
    ax.set_xlabel('Trial time (s)')


In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    plot_speed_def(ax)

---

plot the speed correlation plot for LATE animals

In [ ]:
def late_lesion_correlation_with_size(root, ax, Profiles, badAnimals, color, TaskParamToPlot:str,
                                      preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre][~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:][~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    size=[]
    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue
        try:
            Hist=HistologyExcel(NAS_PATH,animal)
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        behav.append(yPost[i]-yPre[i])
        size.append(Hist.lesion_size())
        animals.append(animal)
        
        #plotting
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=4, c=color[tag], label=tag,edgecolors='none',linewidths=.1);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    def _tik(x,pos):
        return ('$%+g$' if x>0 else '$%g$')%x
    
    ax.set_ylim([-42,10])
    ax.spines['left'].set_bounds(-40,10)
    ax.set_yticks([-40,-30,-20,-10,0,10])
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'$\Delta$ {TaskParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    
    a=late_lesion_correlation_with_size(root, ax=ax, Profiles=Profiles, badAnimals=[],color=color, TaskParamToPlot=TaskParamToPlot,
                                      preSlice=preSlice, postSlice=postSlice)

---

plot the speed correlation plot for any animal

In [ ]:
def behav_lesion_size_correlation(root,ax,PerfParamToPlot,AnimalProfile, color,
                                  animalList=None, Ndays=slice(-5,None), **kwargs):

    if animalList is None:
        animalList=batch_get_animal_list(root,AnimalProfile)

    size=[]
    behav=[]
    animals=[]
    for animal in animalList:
        try:
            i=HistologyExcel(NAS_PATH,animal)
        except AssertionError as e:
            logging.warning(repr(e))
            continue
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        

        res=animal_learning_stats(root, animal, PerfParamToPlot, profile=AnimalProfile,
                                  sessionSlice=Ndays, redo=False)
        
        try:
            t1=np.nanmedian(res)
            t3=i.lesion_size()
        except Exception as e:
            logging.warning(repr(e))
            continue
        behav.append(t1)
        size.append(t3)
        animals.append(animal)
    
        #PLOTTING
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=4, c=color[tag], label=tag, **kwargs);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'{PerfParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markerfacecolor='xkcd:gray', marker='o',markeredgewidth=.1,
                                       markeredgecolor='xkcd:black', markersize=2, label='Late')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:black', marker='D',markeredgewidth=.1,
                                       markerfacecolor='xkcd:gray', markersize=2, label='Early')

    leg=ax.legend(handles=[r_marker,b_marker],loc=(0,-.02),mode=None,ncol=3,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False,handletextpad=0,columnspacing=1)
    return leg

def add_legend_for_lesion_groups(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:orange', marker='o',
                                       markerfacecolor='xkcd:orange', markersize=3, label=f'DLS')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='purple', marker='o',
                                       markerfacecolor='purple', markersize=3, label=f'DMS')
    m_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:green', marker='o',
                                       markerfacecolor='xkcd:green', markersize=3, label=f'DS')


    leg=ax.legend(handles=[r_marker,b_marker,m_marker],loc=(0,0),mode='expand',ncol=1,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False, handletextpad=0)
    return leg

In [ ]:
if "__file__" not in dir():
    
    #Define desired profile dictionary
    AnimalProfile={'Type':'Good',
         'option': ['not used'],
         'rewardType':'Progressive',
         'initialSpeed':['10'],
         'Speed':'10',
         'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS']
                  }
    PerfParamToPlot="Forward Running Speed"
    color={'DS':'m','DMS':'b', 'DLS':'r', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    behav, size, animals=behav_lesion_size_correlation(root,ax,PerfParamToPlot,AnimalProfile,color, marker='v')
    
    r,p=stats.pearsonr(size, behav)
    print(r,p)

---

plotting group bar plots

In [ ]:
def plot_event_rec(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials', shift=0,
                   nPre=5, nPost=10, **kwarg):

    if badAnimals is None:
        badAnimals=[]
    
    #getting the data
    animalList,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    SessionDict={animal:SessionDict[animal] for animal in animalList}
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xData=np.append(np.arange(-nPre,0),np.arange(1,nPost+1))

    groupData=np.nanmedian(data,axis=0)
    
    groupErr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(xData + shift, groupData[nSessionPre-nPre:nSessionPre+nPost],
                abs(groupErr-groupData)[:,nSessionPre-nPre:nSessionPre+nPost],
                fmt='-o', zorder=5, ms=2, elinewidth=1,alpha=.8, **kwarg)
    
    def _tik(x,pos):
        if x in [-nPre,-1,1,nPost]:
            return ('$%+g$' if x>0 else '$%g$')%x
        else:
            return ''
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(_tik))
    
    ax.set_xlim([-nPre-.4,nPost+.4])
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xticks(xData)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0,100])
    ax.set_ylabel(TaskParamToPlot)
#     ax.set_xlabel('Sessions relative to break / lesion')

    return data[:,nSessionPre]

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="% good trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    a=plot_event_rec(root, ax, Profiles,color='k',shift=-.1*.25)
    ax.set_ylim([0,100])

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, Profiles, colorCode, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nInit=slice(0,3), nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                    seed=1, animal_plot=True):
    if badAnimals is None:
        badAnimals=[]
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-3*diff,x_c-diff,x_c+diff,x_c+3*diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    animalList,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    
    data=np.empty((len(animalList),4))
    for i,animal in enumerate(animalList):
        preSession,postSession=sessionDict[animal][0], sessionDict[animal][1]
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot,
                                  goodSessions=[*preSession,*postSession],redo=False)
        
        data[i,0]=np.nanmedian(out[:len(preSession)][nInit])
        data[i,1]=np.nanmedian(out[:len(preSession)][nPre])
        data[i,2]=np.nanmedian(out[len(preSession):][nPost])
        data[i,3]=np.nanmedian(out[len(preSession):][nFin])
        
            
    
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='-o', zorder=5, ms=3, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=seed)
        _coeff=5000
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitInit=np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPre =np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)
            jitFin =np.random.uniform(low=x_pos[3]-diff/_coeff, high=x_pos[3]+diff/_coeff, size=1)
            
#             ax.scatter([jitPre,jitFin],data[i,[1,-1]], s=2, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitInit,jitPre,jitPost,jitFin],data[i,:], c='gray', lw=.2, alpha=.4)
#             ax.plot([jitPre,jitFin],data[i,[1,-1]], c=c, lw=.4, alpha=.6)

    ax.set_xlim([x_pos[0]-diff/2,x_pos[-1]+diff/2])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Na\u00EFve','Before','Acute','Stable'])
    ax.xaxis.set_tick_params(rotation=45)
    ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel(TaskParamToPlot)

    return data,animalList

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DMS-Sharp','Late-Lesion_DLS','Late-Lesion_DS-Sharp']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="Front Starting Trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    data,animalList=plot_event_1on1(root, ax, Profiles, colorCode={'Control':'r'}, badAnimals=None, TaskParamToPlot=TaskParamToPlot,
                       x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(6,10),
                        seed=1, animal_plot=True)
    


---

plotting SPEED sketch

In [ ]:
def plot_speed_sketch(ax):
    ax2=ax.twinx()
    twinColor='r'
    origColor='b'
    
    session="Rat304_2018_12_07_13_34"
    trial=28
    data=Data(root,session[:6],session)
    ET=int(data.cameraSamplingRate*(data.entranceTime[trial]+1.5))
    time=np.linspace(-1,data.entranceTime[trial]+1.5,ET)
    position=data.position[trial][:ET]
    speed=smooth(-np.diff(position)*data.cameraSamplingRate,6)
    
    ax.spines['top'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax.set_xticks([0,2,4,6,8])
    ax2.set_ylim([-15,60])
    ax2.set_yticks([-10,0,20,40,60])
    ax.set_ylim([0,90])
    ax.set_yticks([0,30,60,90])
    ax2.spines['right'].set_color(twinColor)
    ax2.tick_params(axis='y', colors=twinColor)
#     ax2.set_ylabel('Speed (cm/s)', color=twinColor)
    ax2.spines['left'].set_color(origColor)
    ax.tick_params(axis='y', colors=origColor)
    ax.set_ylabel('Position (cm)', color=origColor)

    
    
    ax.plot(time,position, color=origColor, lw=1)
    ax2.plot(time[:-1],speed, color=twinColor, lw=1)

In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(2,2),dpi=100)
    ax=fig.add_subplot(111)
    plot_speed_sketch(ax)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'
    
    colorSig='goldenrod'



    
    #===============================================
    
    # GRID 1 PARAMS
    
    _s=.15
    ShiftDLS1 =-1*_s
    ShiftDMS1 =0*_s
    ShiftDS1  =1*_s
    
    nPre1=5
    nPost1=10
    
    TaskParamToPlot1="Forward Running Speed"
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS1=(profilePreLesion1,profileDLS1,)
    ProfilesDMS1=(profilePreLesion1,profileDMS1,)
    ProfilesDS1 =(profilePreLesion1,profileDS1,)

    preSlice1=slice(-2,None)
    postSlice1=slice(0,2)
    finSlice1=slice(8,10)

    
    
    #================================================
    
    # GRID 2 PARAMS

    initSlice2=slice(0,2)
    preSlice2=slice(-2,None)
    postSlice2=slice(0,2)
    finSlice2=slice(8,10)

    
    
    #================================================
    
    # GRID 3 PARAMS

    profileLesions3={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }

    Profiles3=(profilePreLesion1,profileLesions3)
    
    preSlice3=slice(-5,None)
    postSlice3=slice(0,5)
    finSlice3=slice(3,9)
    
    
    #================================================
    
    # GRID 4 PARAMS

    profile4={'Type':'Good',
             'option': ['not used', 'AsymmetricLesion'],
             'rewardType':'Progressive',
             'initialSpeed':['10'],
             'Speed':'10',
             'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                            'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}
    


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(3.5,2.5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    
    ##########################################
    # 0: % Speed Sketch
    gs0= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.72, right=.225, top=1)
    ax0= fig.add_subplot(gs0[0])
        
        
    plot_speed_sketch(ax0)    
        
        
    ##########################################
    # 1: % speed time course
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0.4, bottom=.72, right=1, top=1)
    ax1= fig.add_subplot(gs1[0])
    
    #DLS
    xDLS1=plot_event_rec(root, ax1, ProfilesDLS1,TaskParamToPlot=TaskParamToPlot1,
                         nPre=nPre1, nPost=nPost1, color=DLSColor,shift=ShiftDLS1)

    #DMS
    xDMS1=plot_event_rec(root, ax1, ProfilesDMS1,TaskParamToPlot=TaskParamToPlot1,
                         nPre=nPre1, nPost=nPost1, color=DMSColor,shift=ShiftDMS1)

    #DS
    xDS1 =plot_event_rec(root, ax1, ProfilesDS1,TaskParamToPlot=TaskParamToPlot1,
                         nPre=nPre1, nPost=nPost1, color=DSColor,shift=ShiftDS1)

    
    
    ax1.set_ylim([25,93])
    ax1.spines['left'].set_bounds(30,90)
    ax1.set_yticks(np.arange(30,91,10))
    ax1.set_yticklabels([30,'',50,'',70,'',90])
    ax1.set_ylabel('Speed (cm/s)')
    ax1.set_xlabel('Session # relative to lesion', labelpad=0)

    plot_session_def(ax1,sessionSlices=(preSlice1,postSlice1,))
    plot_session_def(ax1,sessionSlices=(finSlice1,))

    
    
    
    ##########################################
    # 2: speed event plot
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=.1, bottom=.1, right=.36, top=.5)
    ax2= fig.add_subplot(gs2[0])
        
    data2,*_=plot_event_1on1(root, ax2, Profiles3, colorCode=ColorCode, badAnimals=None, x_pos=(1,2,3,4),
                         TaskParamToPlot=TaskParamToPlot1, nInit=initSlice2, nPre=preSlice2, nPost=postSlice2, nFin=finSlice2)


    ax2.set_ylim([25,93])
    ax2.spines['left'].set_bounds(30,90)
    ax2.set_yticks(np.arange(30,91,10))
    ax2.set_yticklabels([30,'',50,'',70,'',90])
    ax2.set_ylabel('Speed (cm/s)')


    #STATS
    p2,s2=bootstrapTest(data2[:,-1]-data2[:,1])
    permtest.plotSigPair(ax2,y=90,x=(2,4), s=s2, color=colorSig,lw=.8)
    
    
    
        
    ##########################################
    # 3: % speed correlation
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=.5, bottom=.1, right=.9, top=.5)
    ax3= fig.add_subplot(gs3[0])
  
    behav3, size3, animals3=late_lesion_correlation_with_size(root, ax=ax3, Profiles=Profiles3, badAnimals=[],
                                                              color=ColorCode,
                                                              TaskParamToPlot=TaskParamToPlot1,
                                                              preSlice=preSlice3, postSlice=finSlice3)
    
    
    add_legend_for_lesion_groups(ax3)
    
    r3,p3=stats.pearsonr(size3, behav3)
    s=f'$n={len(animals3)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r3)+'\n'+'$p=$'+'{}'.format(SciNote(p3))
    ax3.text(1,10,s, ha='right',va='top', fontsize=5)
    
    ax3.set_ylabel('$\Delta$Speed (cm/s)')
    
    
    
    
    
    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax0,ax1, ax2,ax3)
    OFFX=np.array([.09]*len(AXES))
    OFFY=np.array([.03]*len(AXES))
    OFFX[[-1]]=0.12
    OFFX[[1]]=0.06
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Speed.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()